In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as ptch
%matplotlib inline
import pandas as pd
import ipywidgets as widgets
from ipywidgets import interact

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.display.float_format = '{:.16f}'.format

$f(x) = x\sin(x)\cos(x)$

In [2]:
def F(x: float) -> float:
    return x * np.cos(x) * np.sin(x)

$f'(x) = x \cos^2(x) + \cos(x) \sin(x) - x \sin^2(x)$

In [3]:
def dF(x: float) -> float:
    return x * np.cos(x)**2 + np.cos(x) * np.sin(x) - x * np.sin(x)**2

$f''(x) = 2(\cos(2x) - x\sin(2x))$

In [4]:
def ddF(x: float) -> float:
    return 2 * (np.cos(2 * x) - x * np.sin(2 * x))

$f^{(4)}(x) = 8(x\sin(2x) - 2\cos(2x))$

In [5]:
def d4F(x: float) -> float:
    return 8 * (x * np.sin(2 * x) - 2 * np.cos(2 * x))

$\displaystyle\int f(x)\ dx = \frac{1}{8}(\sin(2x) - 2x\cos(2x)) + c$

In [6]:
def IF(x: float) -> float:
    return (np.sin(2 * x) - 2 * x * np.cos(2 * x)) / 8

$p_0(x) = 2$


In [7]:
def P0(x: float) -> float:
    return 2

$\displaystyle\int p_0(x)\ dx = 2x + c$ 

In [8]:
def IP0(x: float) -> float:
    return 2 * x

$p_1(x) = x+1$


In [9]:
def P1(x: float) -> float:
    return x + 1

$p_1'(x) = 1$

In [10]:
def dP1(x: float) -> float:
    return 1

$\displaystyle\int p_1(x)\ dx = \frac{x^2}{2} + x + c$

In [11]:
def IP1(x: float) -> float:
    return x ** 2 / 2 + x

$p_2(x) = 3x^2 - 2$

In [12]:
def P2(x: float) -> float:
    return 3 * x ** 2 - 2

$p_2'(x) = 6x$

In [13]:
def dP2(x: float) -> float:
    return 6 * x

$p_2''(x) = 6$

In [14]:
def ddP2(x: float) -> float:
    return 6

$\displaystyle\int p_2(x)\ dx = x^3 - 2x + c$

In [15]:
def IP2(x: float) -> float:
    return x ** 3 - 2 * x

$p_3(x) = 4x^3 - 3x^2 + 2x$

In [16]:
def P3(x: float) -> float:
    return 4 * x ** 3 - 3 * x ** 2 + 2 * x

$p_3'(x) = 12x^2 - 6x + 2$

In [17]:
def dP3(x: float) -> float:
    return 12 * x**2 - 6 * x + 2

$p_3''(x) = 24x - 6$

In [18]:
def ddP3(x: float) -> float:
    return 24 * x - 6

$\displaystyle\int p_3(x)\ dx = x^4 - x^3 + x^2$

In [19]:
def IP3(x: float) -> float:
    return x ** 4 - x ** 3 + x ** 2

Адаптированные квадратурные формулы

In [20]:
def getSum(f, a: float, b: float, n: int) -> float:
    h = (b - a) / n
    return sum([f(a + i * h) for i in range(1, n)])


def KFL(f, s: float, a: float, b: float, n: int) -> float:
    return (f(a) + s) * (b - a) / n


def KFR(f, s: float, a: float, b: float, n: int) -> float:
    return (s + f(b)) * (b - a) / n


def KFM(f, s: float, a: float, b: float, n: int) -> float:
    h = (b - a) / n
    return sum([f(a + i * h + h / 2) for i in range(n)]) * (b - a) / n


def KFT(f, s: float, a: float, b: float, n: int) -> float:
    return (f(a) + 2 * s + f(b)) * (b - a) / (2 * n)


def KFS(f, s: float, a: float, b: float, n: int) -> float:
    h = (b - a) / n
    sMid = sum([f(a + (2 * i + 1) * h / 2) for i in range(n)])
    return (f(a) + 2 * s + 4 * sMid + f(b)) * h / 6

Функции для нахождения теоретической погрешности

In [21]:
def zero(x: float) -> float:
    return 0


def RKFL(df, a: float, b: float, n: int) -> float:
    X = np.linspace(a, b, 1000)
    return (b - a)**2 * max([abs(df(x)) for x in X]) / (2 * n)


def RKFR(df, a: float, b: float, n: int) -> float:
    X = np.linspace(a, b, 1000)
    return (b - a)**2 * max([abs(df(x)) for x in X]) / (2 * n)


def RKFM(ddf, a: float, b: float, n: int) -> float:
    X = np.linspace(a, b, 1000)
    return (b - a)**3 * max([abs(ddf(x)) for x in X]) / (24 * n**2)


def RKFT(ddf, a: float, b: float, n: int) -> float:
    X = np.linspace(a, b, 1000)
    return (b - a)**3 * max([abs(ddf(x)) for x in X]) / (12 * n**2)


def RKFS(d4f, a: float, b: float, n: int) -> float:
    X = np.linspace(a, b, 1000)
    return (b - a)**5 * max([abs(d4f(x)) for x in X]) / (2880 * n**4)

Элементы для вывода

In [22]:
def printGraph(f, a: float, b: float, expected: float, n: int, name: str, res: float):
    
    ptsnum = 1000
    ep = (b - a) / 8

    X = np.linspace(a - ep, b + ep, ptsnum)
    y = [f(x) for x in X]
    Ox = [0 for _ in X]

    fig, ax = plt.subplots()
    ax.plot(X, y, color="blue", alpha=0.5, label=f"{f.__name__}(x)")
    ax.plot(X, Ox, color="black")

    X = [x for x in X if a <= x <= b]
    y = [f(x) for x in X]
    Xaxis = [0 for _ in X]

    h = (b - a) / n

    figures = []

    if (name == "KFL"):
        for i in range(n):
            c = a + i * (b - a) / n
            elem = ((c, 0), (c, f(c)), (c+h, f(c)), (c+h, 0))
            figures.append(elem)
    elif (name == "KFR"):
        for i in range(n):
            c = a + i * (b - a) / n
            elem = ((c, 0), (c, f(c + h)), (c+h, f(c + h)), (c+h, 0))
            figures.append(elem)
    elif (name == "KFM"):
        for i in range(n):
            c = a + i * (b - a) / n
            elem = ((c, 0), (c, f(c + h / 2)), (c+h, f(c + h / 2)), (c+h, 0))
            figures.append(elem)
    elif (name == "KFT"):
        for i in range(n):
            c = a + i * (b - a) / n
            elem = ((c, 0), (c, f(c)), (c+h, f(c + h)), (c+h, 0))
            figures.append(elem)

    ax.add_patch(ptch.Polygon(figures[0], edgecolor="#1207ff", facecolor="#1207ff", 
                                        fill=True, alpha=0.2, label=f"{round(res, 3)}"))       

    figures.remove(figures[0])
    
    [ax.add_patch(ptch.Polygon(figure, edgecolor="#1207ff", facecolor="#1207ff", 
                                        fill=True, alpha=0.2)) for figure in figures]

    ax.fill_between(X, Xaxis, y, facecolor="red", alpha=0.2, label=f"{round(expected, 3)}")
    ax.vlines(a, min(f(a), 0), max(f(a), 0), color="blue", alpha=0.2, linestyle="--")
    ax.vlines(b, min(f(b), 0), max(f(b), 0), color="blue", alpha=0.2, linestyle="--")
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.legend()

    plt.show()


def getGraph(f, If, formula, a: float, b: float, n: int):
    Sum = getSum(f, a, b, n)
    Res = formula(f, Sum, a, b, n)
    Expected = If(b) - If(a)
    printGraph(f, a, b, Expected, n, formula.__name__, Res)


def getGraphInteract(f, If, formula, a, b, n__):
    def graph_(n=n__):
        return getGraph(f, If, formula, a, b, n)
    return graph_


def getData(f, If, df, ddf, d4f, a: float, b: float, n: int):
    Sum = getSum(f, a, b, n)
    Expected = If(b) - If(a)
    Results = [If(b) - If(a), KFL(f, Sum, a, b, n), KFR(f, Sum, a, b, n), 
                        KFM(f, Sum, a, b, n), KFT(f, Sum, a, b, n), KFS(f, Sum, a, b, n)]
    R = [0, RKFL(df, a, b, n), RKFR(df, a, b, n), RKFM(ddf, a, b, n), RKFT(ddf, a, b, n), RKFS(d4f, a, b, n)]
    data = {
        "Значение": Results,
        "Фактическая Погрешность": [abs(Expected - result) for result in Results],
        "Теоретическая Погрешность": R
    }
    df = pd.DataFrame(data, index=["Точный интеграл", "СКФ левых прямоугольников", 
                        "СКФ правых прямоугольников", "СКФ средних прямоугольников",
                        "СКФ трапеций", "СКФ Симпсона"])
    print(df)


Входные данные:

In [23]:
promptA = widgets.FloatText(
    value=-6,
    description='a: ',
    disabled=False
)
promptB = widgets.FloatText(
    value=6,
    description='b: ',
    disabled=False
)
promptN = widgets.IntText(
    value=20,
    description='n: ',
    disabled=False
)
display(promptA)
display(promptB)
display(promptN)

FloatText(value=-6.0, description='a: ')

FloatText(value=6.0, description='b: ')

IntText(value=20, description='n: ')

In [34]:
a_ = promptA.value
b_ = promptB.value
n_ = promptN.value
print(f"a: {a_}\nb: {b_}\nn: {n_}")

a: 0.0
b: 10.0
n: 40


Визуализация СКФ левых прямоугольников для $f(x)$

In [35]:
graph = getGraphInteract(F, IF, KFL, a_, b_, n_)
interact(graph, n=(2, n_ * 2, 1)) 
 

interactive(children=(IntSlider(value=40, description='n', max=80, min=2), Output()), _dom_classes=('widget-in…

<function __main__.getGraphInteract.<locals>.graph_(n=40)>

Визуализация СКФ правых прямоугольников для $f(x)$

In [36]:
graph = getGraphInteract(F, IF, KFR, a_, b_, n_)
interact(graph, n=(2, n_ * 2, 1)) 

interactive(children=(IntSlider(value=40, description='n', max=80, min=2), Output()), _dom_classes=('widget-in…

<function __main__.getGraphInteract.<locals>.graph_(n=40)>

Визуализация СКФ средних прямоугольников для $f(x)$

In [37]:
graph = getGraphInteract(F, IF, KFM, a_, b_, n_)
interact(graph, n=(2, n_ * 2, 1)) 

interactive(children=(IntSlider(value=40, description='n', max=80, min=2), Output()), _dom_classes=('widget-in…

<function __main__.getGraphInteract.<locals>.graph_(n=40)>

Визуализация СКФ трапеций для $f(x)$

In [38]:
graph = getGraphInteract(F, IF, KFT, a_, b_, n_)
interact(graph, n=(2, n_ * 2, 1)) 

interactive(children=(IntSlider(value=40, description='n', max=80, min=2), Output()), _dom_classes=('widget-in…

<function __main__.getGraphInteract.<locals>.graph_(n=40)>

Значения СКФ, их абсолютная фактическая и теоретическая погрешность для $f(x)$

In [39]:
getData(F, IF, dF, ddF, d4F, a_, b_, n_)

                                       Значение  Фактическая Погрешность  \
Точный интеграл             -0.9060869981925264       0.0000000000000000   
СКФ левых прямоугольников   -1.4529269386287691       0.5468399404362427   
СКФ правых прямоугольников  -0.3117453752192347       0.5943416229732916   
СКФ средних прямоугольников -0.9180071611889609       0.0119201629964345   
СКФ трапеций                -0.8823361569240019       0.0237508412685244   
СКФ Симпсона                -0.9061168264339746       0.0000298282414483   

                             Теоретическая Погрешность  
Точный интеграл                     0.0000000000000000  
СКФ левых прямоугольников          11.8467480529561691  
СКФ правых прямоугольников         11.8467480529561691  
СКФ средних прямоугольников         0.4566396144740068  
СКФ трапеций                        0.9132792289480136  
СКФ Симпсона                        0.0009754995292172  


Значения СКФ, их абсолютная фактическая и теоретическая погрешность для $p_0(x)$

In [40]:
getData(P0, IP0, zero, zero, zero, a_, b_, n_)

                                       Значение  Фактическая Погрешность  \
Точный интеграл             20.0000000000000000       0.0000000000000000   
СКФ левых прямоугольников   20.0000000000000000       0.0000000000000000   
СКФ правых прямоугольников  20.0000000000000000       0.0000000000000000   
СКФ средних прямоугольников 20.0000000000000000       0.0000000000000000   
СКФ трапеций                20.0000000000000000       0.0000000000000000   
СКФ Симпсона                20.0000000000000000       0.0000000000000000   

                             Теоретическая Погрешность  
Точный интеграл                     0.0000000000000000  
СКФ левых прямоугольников           0.0000000000000000  
СКФ правых прямоугольников          0.0000000000000000  
СКФ средних прямоугольников         0.0000000000000000  
СКФ трапеций                        0.0000000000000000  
СКФ Симпсона                        0.0000000000000000  


Значения СКФ, их абсолютная фактическая и теоретическая погрешность для $p_1(x)$

In [41]:
getData(P1, IP1, dP1, zero, zero, a_, b_, n_)

                                       Значение  Фактическая Погрешность  \
Точный интеграл             60.0000000000000000       0.0000000000000000   
СКФ левых прямоугольников   58.7500000000000000       1.2500000000000000   
СКФ правых прямоугольников  61.2500000000000000       1.2500000000000000   
СКФ средних прямоугольников 60.0000000000000000       0.0000000000000000   
СКФ трапеций                60.0000000000000000       0.0000000000000000   
СКФ Симпсона                60.0000000000000000       0.0000000000000000   

                             Теоретическая Погрешность  
Точный интеграл                     0.0000000000000000  
СКФ левых прямоугольников           1.2500000000000000  
СКФ правых прямоугольников          1.2500000000000000  
СКФ средних прямоугольников         0.0000000000000000  
СКФ трапеций                        0.0000000000000000  
СКФ Симпсона                        0.0000000000000000  


Значения СКФ, их абсолютная фактическая и теоретическая погрешность для $p_2(x)$

In [42]:
getData(P2, IP2, dP2, ddP2, zero, a_, b_, n_)

                                         Значение  Фактическая Погрешность  \
Точный интеграл              980.0000000000000000       0.0000000000000000   
СКФ левых прямоугольников    942.8125000000000000      37.1875000000000000   
СКФ правых прямоугольников  1017.8125000000000000      37.8125000000000000   
СКФ средних прямоугольников  979.8437500000000000       0.1562500000000000   
СКФ трапеций                 980.3125000000000000       0.3125000000000000   
СКФ Симпсона                 980.0000000000000000       0.0000000000000000   

                             Теоретическая Погрешность  
Точный интеграл                     0.0000000000000000  
СКФ левых прямоугольников          75.0000000000000000  
СКФ правых прямоугольников         75.0000000000000000  
СКФ средних прямоугольников         0.1562500000000000  
СКФ трапеций                        0.3125000000000000  
СКФ Симпсона                        0.0000000000000000  


Значения СКФ, их абсолютная фактическая и теоретическая погрешность для $p_3(x)$

In [43]:
getData(P3, IP3, dP3, ddP3, zero, a_, b_, n_)

                                         Значение  Фактическая Погрешность  \
Точный интеграл             9100.0000000000000000       0.0000000000000000   
СКФ левых прямоугольников   8640.9375000000000000     459.0625000000000000   
СКФ правых прямоугольников  9570.9375000000000000     470.9375000000000000   
СКФ средних прямоугольников 9097.0312500000000000       2.9687500000000000   
СКФ трапеций                9105.9375000000000000       5.9375000000000000   
СКФ Симпсона                9100.0000000000000000       0.0000000000000000   

                             Теоретическая Погрешность  
Точный интеграл                     0.0000000000000000  
СКФ левых прямоугольников        1427.5000000000000000  
СКФ правых прямоугольников       1427.5000000000000000  
СКФ средних прямоугольников         6.0937500000000000  
СКФ трапеций                       12.1875000000000000  
СКФ Симпсона                        0.0000000000000000  
